<a href="https://colab.research.google.com/github/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting/blob/main/notebooks/n_beats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

from google.colab import userdata
token = userdata.get('GITHUB_TOKEN')
user_name = userdata.get('GITHUB_USERNAME')
mail = userdata.get('GITHUB_MAIL')

!git config --global user.name "{user_name}"
!git config --global user.email "{mail}"
!git clone https://{token}@github.com/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting.git

%cd Walmart-Recruiting-Store-Sales-Forecasting

from google.colab import userdata
! pip install -r ./requirements.txt
kaggle_json_path = userdata.get('KAGGLE_JSON_PATH')
! ./src/data_loader.sh -f {kaggle_json_path}

# **Read Data**

In [36]:
# **Torch**

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
import torch
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
from neuralforecast.losses.pytorch import MSE

from src.config import *

stores = pd.read_csv(STORES_PATH)
features = pd.read_csv(FEATURES_PATH)
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

from src import data_loader, processing
import importlib
importlib.reload(processing)

dataframes = data_loader.load_raw_data()
df = processing.run_preprocessing(dataframes, process_test=False, merge_features=False, merge_stores=False)['train']
X_train, y_train, X_valid, y_valid = processing.split_data_by_ratio(df, separate_target=True)

print(f"Shapes of train_df and valid_df: {X_train.shape}, {X_valid.shape}")

Data loading complete.
Shapes of train_df and valid_df: (337256, 4), (84314, 4)


# **Custom NBEATS**

In [20]:
class CustomNBEATS(NBEATS):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.9)

    def set_optim(self,optimizer):
      self.optimizer = optimizer
      return self

    def set_scheduler(self,scheduler):
      self.scheduler = scheduler
      return self

    def configure_optimizers(self):
        # Define your custom optimizer here
        optimizer = self.optimizer #torch.optim.AdamW(self.parameters(), lr=1e-3)

        # Optional: add scheduler if needed
        scheduler = self.scheduler #torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'interval': 'epoch',
                'frequency': 1
            }
        }

In [32]:
from itertools import product
from neuralforecast.models import PatchTST
from models.neural_forecast_models import NeuralForecastModels
from src.utils import wmae as compute_wmae
import logging

logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("neuralforecast").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
logging.getLogger("lightning_fabric").setLevel(logging.WARNING)

def run_nbeats_cv(X_train, y_train, X_valid, y_valid,
                            param_grid,
                            fixed_params,
                            return_all=False):
    results = []

    keys, values = zip(*param_grid.items())
    for vals in product(*values):
        params = dict(zip(keys, vals))
        params.update(fixed_params)

        params['enable_progress_bar'] = False
        params['enable_model_summary'] = False

        model = NBEATS(**params)

        nf_model = NeuralForecastModels(models=[model], model_names=['NBEATS'], freq='W-FRI', one_model=True)
        nf_model.fit(X_train, y_train)
        y_pred = nf_model.predict(X_valid)
        score = compute_wmae(y_valid, y_pred, X_valid['IsHoliday'])

        result = {'wmae': score, 'preds': y_pred}
        result.update(params)

        results.append(result)
        print(" → ".join(f"{k}={v}" for k,v in params.items() if k not in ['enable_progress_bar','enable_model_summary']) + f" → WMAE={score:.4f}")

    if return_all:
        return results
    else:
        return min(results, key=lambda r: r['wmae'])

In [39]:
param_grid = {
    'activation': ['LeakyReLU','ReLU', 'Tanh','PReLU'],
    # 'stride': [1, 2, 4],
    # 'input_size' : [40,52,60,72]
}

fixed_params = {
    'max_steps': 25 * 104,
    'h': 53,
    'random_seed': 42,
    'input_size': 52,
    'batch_size' : 64,
}

best_result = run_nbeats_cv(
    X_train, y_train, X_valid, y_valid,
    param_grid=param_grid,
    fixed_params=fixed_params,
    return_all=False
)

print("\nBest hyperparameters found:")
for param in param_grid.keys():
    print(f"  {param}: {best_result[param]}")
print(f"Best WMAE: {best_result['wmae']:.4f}")

activation=LeakyReLU → max_steps=2600 → h=53 → random_seed=42 → input_size=52 → batch_size=64 → WMAE=1602.8075
activation=ReLU → max_steps=2600 → h=53 → random_seed=42 → input_size=52 → batch_size=64 → WMAE=1593.9089
activation=Tanh → max_steps=2600 → h=53 → random_seed=42 → input_size=52 → batch_size=64 → WMAE=2985.8828
activation=PReLU → max_steps=2600 → h=53 → random_seed=42 → input_size=52 → batch_size=64 → WMAE=1629.1809

Best hyperparameters found:
  activation: ReLU
Best WMAE: 1593.9089


**Cross Validation**

In [ ]:
! wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: azhgh22 (MLBeasts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [42]:
from models.neural_forecast_models import NeuralForecastModels
from src.utils import wmae as compute_wmae
import logging
from sklearn.pipeline import Pipeline

logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("neuralforecast").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
logging.getLogger("lightning_fabric").setLevel(logging.WARNING)

models = [NBEATS(
    h=53,
    input_size=52,
    # n_stacks=4,
    # nb_blocks_per_stack=4,
    # hidden_size=512,
    # dropout_prob=0.1,
    # stack_types=['trend', 'seasonality'],
    activation='ReLU',
    # share_weights_in_stack=True,
    max_steps=5000,
    batch_size=64,
    learning_rate=1e-3,
    random_seed=42
)]


pipeline = Pipeline([
    ('model', NeuralForecastModels(models,['NBEATS']))
])

model = pipeline.fit(X_train, y_train)
y_pred = model.predict(X_valid)
score = compute_wmae(y_valid, y_pred, X_valid['IsHoliday'])

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


TypeError: unsupported operand type(s) for -: 'float' and 'dict'

In [43]:
y_pred

{'NBEATS': 0         5171.333984
 1         7471.615723
 2        23447.093750
 3         5501.274902
 4         2284.924316
              ...     
 84309     1573.205078
 84310     4370.434570
 84311    51356.046875
 84312     5750.613770
 84313      818.661316
 Name: NBEATS, Length: 84314, dtype: float32}

In [ ]:
import wandb
import joblib
param_grid = {
    'activation': ['LeakyReLU','ReLU', 'Tanh','PReLU'],
    # 'stride': [1, 2, 4],
    # 'input_size' : [40,52,60,72]
}

fixed_params = {
    'max_steps': 25 * 104,
    'h': 53,
    'random_seed': 42,
    'input_size': 52,
    'batch_size' : 64,
}

fin_model = pipeline.fit(train.drop(columns=['Weekly_Sales'].copy()),train['Weekly_Sales'].copy())

joblib.dump(fin_model, "nbeats_run3.pkl")
wandb.init(project="Walmart Recruiting - Store Sales Forecasting", name="nbeats:run3")

wandb.config.update({
    'score_metric' : 'WMAE',
    'score_policy' : {
        'weight on holidays' : 5,
        'weight on non_holidays' : 1
    },
    'model' : 'nbeats',
    'learning_rate' : 0.001,
    'learning_scheduler' : 'True',
    'learning_scheduler_step' : 10,
    'learning_scheduler_gamma' : 0.75,
    'weight_decay' : 0,
    'batch_size' : 64,
    'max_depth' : 7,
    'max_steps' : 25 * 104,
    'input_size' : 15,
    'horizon': 48,
    'architecture' : ['identity', 'trend', 'seasonality'],
    'n_blocks' : [1,1,1],
    'random_state': 42,
    'objective' : 'reg:squarederror',
})

wandb.log({
    'val_wmae': score
})


artifact = wandb.Artifact(
    name="nbeats_run3",
    type="model",
)

artifact.add_file("nbeats_run3.pkl")
wandb.log_artifact(artifact)

wandb.finish()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.
wandb: Currently logged in as: azhgh22 (MLBeasts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


val_wmae,▁
val_wmae,2344.6255


In [ ]:
pred = fin_model.predict(test)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [ ]:
pred

,unique_id,ds,y,IsHoliday,CustomNBEATS
0,1_1,2012-11-02,0,0,31145.664062
1,1_1,2012-11-09,0,0,22807.623047
2,1_1,2012-11-16,0,0,20034.949219
3,1_1,2012-11-23,0,1,19596.896484
4,1_1,2012-11-30,0,0,19121.177734
...,...,...,...,...,...
115059,45_98,2013-06-28,0,0,824.601746
115060,45_98,2013-07-05,0,0,832.055359
115061,45_98,2013-07-12,0,0,831.456604
115062,45_98,2013-07-19,0,0,753.761353


In [ ]:
test1 = test.copy()
test1['unique_id'] = test1["Store"].astype(str) + "_" + test1["Dept"].astype(str)
test1['ds'] = pd.to_datetime(test1['Date'])

In [ ]:
merged = pd.merge(test1, pred, on=['unique_id','ds'],how='left')
merged['Id'] = test1["unique_id"].astype(str) + "_" + test1["ds"].astype(str)
merged['Weekly_Sales'] = merged['CustomNBEATS']

In [ ]:
merged[['Id','Weekly_Sales']].to_csv('submission.csv', index=False)